<a href="https://colab.research.google.com/github/hiten016/Finetuning-techniques/blob/main/Finetuning_Qlora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#Supervised Finetune-> method to improve and customise pre trained LLMS

In [5]:
#improve and customise pre-trained LLM. Retraining the base models on a smaller datatset
#Basically we are training to predict text to follow instructions- >adapt to specific task like PDF generation

In [6]:
#Finetuning improves performance->Qwen2.5 coder 32B (coding tasks)
#SFT Techniques ->
'''
Full Finetuning-> Pretraining all the parameters (Not feasible)
Low Rank adaptation LoRA -> parameter efficient finetuning technique
(Low ranked matrices at each target layer )
Qlora -> Quantisation aware Low ranked adaptation (most efficient)
'''

'\nFull Finetuning-> Pretraining all the parameters (Not feasible)\nLow Rank adaptation LoRA -> parameter efficient finetuning technique\n(Low ranked matrices at each target layer )\nQlora -> Quantisation aware Low ranked adaptation (most efficient)\n'

In [7]:
#unsloth library-> llama 3.2 3B model
#Qlora finetuning on finetome-100k dataset
#Hugging face dataset preferred

In [1]:
pip install unsloth transformers trl

In [5]:
import torch
from unsloth import FastLanguageModel
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth.chat_templates import get_chat_template

ModuleNotFoundError: No module named 'transformers.models.gemma2'

In [ ]:
model, tokenizer =FastLanguageModel.from_pretrained(
        model_name="unsloth/Llama-3.2-3B-Instruct",
        max_seq_length=2048,
        load_in_4bit=True

)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model, r=16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
)

In [ ]:
tokenizer = get_chat_template(tokenizer, chat_template="llama-3.1")

In [ ]:
dataset = load_dataset("mlabonne/FineTome-100k", split="train")

In [ ]:
from unsloth import standardize_sharegpt
dataset = standardize_sharegpt(dataset)

In [ ]:
dataset

In [ ]:
dataset[0]

In [ ]:
dataset = dataset.map(
    lambda examples: {
        "text": [
            tokenizer.apply_chat_template(convo, tokenize=False)
            for convo in examples["conversations"]
        ]
    },
    batched=True
)

In [14]:
dataset

Dataset({
    features: ['conversations', 'source', 'score', 'text'],
    num_rows: 100000
})

In [15]:
dataset[0]

{'conversations': [{'content': 'Explain what boolean operators are, what they do, and provide examples of how they can be used in programming. Additionally, describe the concept of operator precedence and provide examples of how it affects the evaluation of boolean expressions. Discuss the difference between short-circuit evaluation and normal evaluation in boolean expressions and demonstrate their usage in code. \n\nFurthermore, add the requirement that the code must be written in a language that does not support short-circuit evaluation natively, forcing the test taker to implement their own logic for short-circuit evaluation.\n\nFinally, delve into the concept of truthiness and falsiness in programming languages, explaining how it affects the evaluation of boolean expressions. Add the constraint that the test taker must write code that handles cases where truthiness and falsiness are implemented differently across different programming languages.',
   'role': 'user'},
  {'content': 

In [4]:
trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=60,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        output_dir="outputs"
    ),
)

NameError: name 'SFTTrainer' is not defined

In [ ]:
trainer.train()

In [26]:
model.save_pretrained("finetuned_model")

In [27]:
inference_model, inference_tokenizer = FastLanguageModel.from_pretrained(
    model_name="./finetuned_model",
    max_seq_length=2048,
    load_in_4bit=True
)

==((====))==  Unsloth 2025.6.12: Fast Llama patching. Transformers: 4.53.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [28]:
text_prompts = [
    "What are the key principles of investment?"
]

for prompt in text_prompts:
  formatted_prompt = inference_tokenizer.apply_chat_template([{
      "role": "user",
      "content": prompt
      }], tokenize=False)

  model_inputs = inference_tokenizer(formatted_prompt, return_tensors="pt").to("cuda")
  generated_ids = inference_model.generate(
      **model_inputs,
      max_new_tokens=512,
      temperature=0.7,
      do_sample=True,
      pad_token_id=inference_tokenizer.pad_token_id
  )
  response = inference_tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
  print(response)

LlamaForCausalLM has no `_prepare_4d_causal_attention_mask_with_cache_position` method defined in its base modeling class. Compiled forward passes will be sub-optimal. If you're writing code, see Llama for an example implementation. If you're a user, please report this issue on GitHub.


system

Cutting Knowledge Date: December 2023
Today Date: 05 Jul 2025

user

What are the key principles of investment?assistant

The key principles of investment can be summarized as follows:

1. **Diversification**: Spread investments across different asset classes, sectors, and geographic regions to minimize risk and maximize returns.
2. **Risk Management**: Assess and manage risk by understanding the volatility of investments, diversifying, and hedging against potential losses.
3. **Dollar-Cost Averaging**: Invest a fixed amount of money at regular intervals, regardless of market conditions, to reduce the impact of market volatility.
4. **Long-term Focus**: Invest for the long term, as short-term market fluctuations are typically less significant than long-term trends.
5. **Low-Cost Investing**: Minimize investment costs by choosing low-cost index funds, ETFs, and avoiding unnecessary fees.
6. **Tax Efficiency**: Consider tax implications of investments and aim to minimize tax liab